# Under Sampling

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Delay"]

In [ ]:
data = pd.read_csv('/Users/msml/Vamsi/SF/SF_Projects/fiiiiiinallll.csv')
data

In [ ]:
data.isnull().values.any()

In [ ]:
#Create independent and Dependent Features
labels = np.asarray(data["ArrDel15"])
data_select = data.drop(["Quarter", "Year","ArrTime", "DepTime", "Month", "DayofMonth", "ArrDelayMinutes","ArrDel15","DepDel15", "Date", "Airport_y","Airport_x"],axis=1)
lenc = LabelEncoder()
intenc = lenc.fit_transform(data_select["DestAirportID"])
data_select["DestAirportID"] = intenc
intenc = lenc.fit_transform(data_select["OriginAirportID"])
data_select["OriginAirportID"] = intenc
intenc = lenc.fit_transform(data_select["winddirDegree_x"])
data_select["winddirDegree_x"] = intenc
intenc = lenc.fit_transform(data_select["weatherCode_x"])
data_select["weatherCode_x"] = intenc
intenc = lenc.fit_transform(data_select["winddirDegree_y"])
data_select["winddirDegree_y"] = intenc
intenc = lenc.fit_transform(data_select["weatherCode_y"])
data_select["weatherCode_y"] = intenc
features = np.asarray(data_select)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features,labels, test_size=0.30, random_state=42)

In [ ]:
state = np.random.RandomState(42)
X = features_train
Y = labels_train.astype(np.uint8)
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

In [ ]:
count_classes = pd.value_counts(labels_train, sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Transaction Class Distribution")
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency")

In [ ]:
## Get the delay and the normal dataset 
delay = [(i) for i in labels_train if i==1]
normal = [(i) for i in labels_train if i==0]
print(len(delay),len(normal))

In [ ]:
from imblearn.under_sampling import NearMiss
# Implementing Undersampling for Handling Imbalanced 
nm = NearMiss()
X_res,y_res=nm.fit_sample(X,Y)

In [ ]:
X_res.shape,y_res.shape

In [ ]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

In [ ]:
df=pd.DataFrame(X_res)
df

In [ ]:
count_classes = pd.value_counts(y_res, sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Transaction Class Distribution")
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency")

# Over Sampling

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Delay"]

In [ ]:
data = pd.read_csv('/Users/msml/Vamsi/SF/SF_Projects/fiiiiiinallll.csv')
data

In [ ]:
#Create independent and Dependent Features
labels = np.asarray(data["ArrDel15"])
data_select = data.drop(["Quarter", "Year","ArrTime", "DepTime", "Month", "DayofMonth", "ArrDelayMinutes","ArrDel15","DepDel15", "Date", "Airport_y","Airport_x"],axis=1)
lenc = LabelEncoder()
intenc = lenc.fit_transform(data_select["DestAirportID"])
data_select["DestAirportID"] = intenc
intenc = lenc.fit_transform(data_select["OriginAirportID"])
data_select["OriginAirportID"] = intenc
intenc = lenc.fit_transform(data_select["winddirDegree_x"])
data_select["winddirDegree_x"] = intenc
intenc = lenc.fit_transform(data_select["weatherCode_x"])
data_select["weatherCode_x"] = intenc
features = np.asarray(data_select)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features,labels, test_size=0.30, random_state=42)

In [ ]:
state = np.random.RandomState(42)
X = features_train
Y = labels_train.astype(np.uint8)
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

In [ ]:
count_classes = pd.value_counts(labels_train, sort = True)
count_classes.plot(kind = 'bar', rot=0)
plt.title("Transaction Class Distribution")
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency")

In [ ]:
## Get the delay and the normal dataset 
delay = data[data['ArrDel15']==1]
normal = data[data['ArrDel15']==0]
print(delay.shape,normal.shape)

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
os =  RandomOverSampler()

In [ ]:
X_train_res, y_train_res = os.fit_sample(X, Y)

In [ ]:
X_train_res.shape,y_train_res.shape

In [ ]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))